In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import GradientBoostingClassifier

import sys
sys.path.insert(0, '../fairml')
import plotting
import generate as G
import models
import actions
import utils

# Generate the data

In [ ]:
# get the scalers, and the convenince function
x_scaler, z_scaler, generate = G.generate_hmumu()

# generate test data (a large, one time only thing)
n_test_samples = 1000000
X, Y, Z, W = generate(n_test_samples, balanced=False) # need Global weights for testing
Z_plot = z_scaler.inverse_transform(Z, copy=True)

# Create a sklearn benchmark performance

In [ ]:
X_train, Y_train, _, W_train = generate(10000, balanced=True)

# train
gbc100 = GradientBoostingClassifier(n_estimators=100)
gbc100.fit(X_train, Y_train, sample_weight=W_train)
preds100 = gbc100.predict_proba(X)[:, 1]

gbc400 = GradientBoostingClassifier(n_estimators=400)
gbc400.fit(X_train, Y_train, sample_weight=W_train)
preds400 = gbc400.predict_proba(X)[:, 1]

# roc curve
from sklearn.metrics import roc_curve
fpr100, tpr100, _ = roc_curve(Y, preds100, sample_weight=W)
fpr400, tpr400, _ = roc_curve(Y, preds400, sample_weight=W)

# plot
fig, ax = plt.subplots(figsize=(5,5))
ax.plot(1-fpr100, tpr100, label='GBC 100')
ax.plot(1-fpr400, tpr400, label='GBC 400')
ax.legend(loc='best')
ax.set_xlabel('Background rejection')
ax.set_ylabel('Signal efficiency')
plt.show()

# Non-adversarial training of a classifier

In [ ]:
sess = tf.InteractiveSession()
ctr = 0

In [ ]:
n_samples = 1000
n_epochs = 10
learning_rate = 0.005
deep = False
ctr+=1
name = 'name'+str(ctr)

# input placeholders
x_in = tf.placeholder(tf.float32, shape=(None, X.shape[1]), name='X1_X2')
y_in = tf.placeholder(tf.float32, shape=(None, ), name='Y')
z_in = tf.placeholder(tf.float32, shape=(None, ), name='Z')
w_in = tf.placeholder(tf.float32, shape=(None, ), name='W')
inputs = [x_in, y_in, z_in, w_in]

# create the classifier graph, loss, and optimisation
clf_output, vars_D = models.classifier(x_in, name+'_clf', deep=deep)
loss_D = models.classifier_loss(clf_output, y_in, w_in)
opt_D = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_D, var_list=vars_D)

# initialise the variables
sess.run(tf.global_variables_initializer())

# train the classifier
pname = 'Hmumu'
for e in range(n_epochs):
    
    # report on progress
    if e%10 == 0:
        print('{}/{}'.format(e, n_epochs))
    
    # training step
    actions.train(sess, opt_D, loss_D, inputs, generate, n_samples, 1, None)

    # make predictions on the test
    pred = utils.sigmoid(sess.run(clf_output, feed_dict={x_in:X}))

    # name of the plot

    dirn = 'media/plots/{}'.format(pname)
    if not os.path.exists(dirn):
        os.makedirs(dirn)
    path = '{d}/{n}_{c:03}.png'.format(d=dirn, n=pname, c=e)
    
    # plot the performance of the classifier
    fprs = [fpr100, fpr400]
    tprs = [tpr100, tpr400]
    titles = ['GBC100', 'GBC400']
    benchmarks = fprs, tprs, titles
    plotting.plot_hmumu_performance(X, Y, Z_plot, W, pred.reshape(-1), benchmarks, path, batch=True)

if not os.path.exists('media/gifs'):
    os.makedirs('media/gifs')
dirn = 'media/plots/{}'.format(pname)
in_pngs = ' '.join(['{d}/{p}_{c:03}.png'.format(d=dirn, p=pname, c=c) for c in range(n_epochs)])
out_gif = 'media/gifs/{p}_{c}.gif'.format(p=pname, c=n_epochs)
os.system('convert -colors 32 -loop 0 -delay 10 {i} {o}'.format(i=in_pngs, o=out_gif))
print(out_gif)
